In [1]:
import numpy as np, pandas as pd
#import matplotlib.pyplot as plt
import xgboost as xgb
import pickle

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [4]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Impute Item Weight

In [5]:
id_weight =  df.loc[~df.Item_Weight.isna(), ['Item_Identifier', 'Item_Weight']].drop_duplicates()

In [6]:
id_weight

,Item_Identifier,Item_Weight
0,FDA15,9.300
1,DRC01,5.920
2,FDN15,17.500
3,FDX07,19.200
4,NCD19,8.930
...,...,...
7298,NCW05,20.250
7373,FDS09,8.895
7421,FDU43,19.350
7944,FDO49,10.600


In [7]:
df.Item_Identifier.nunique()

1559

In [8]:
df.drop(columns = 'Item_Weight', inplace = True)

In [9]:
df = pd.merge(df, id_weight, 'left', on = 'Item_Identifier')  

In [10]:
df.columns

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales', 'Item_Weight'],
      dtype='str')

In [11]:
df = df[['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales']]

In [12]:
id_weight.to_csv('id_weight.csv', index = False)

Impute Outlet Size

In [13]:
outlet_size = df[~df.Outlet_Size.isna()][['Outlet_Type', 'Outlet_Location_Type', 'Outlet_Size' ]].drop_duplicates().reset_index(drop = True)

In [14]:
outlet_size

,Outlet_Type,Outlet_Location_Type,Outlet_Size
0,Supermarket Type1,Tier 1,Medium
1,Supermarket Type2,Tier 3,Medium
2,Supermarket Type1,Tier 3,High
3,Supermarket Type3,Tier 3,Medium
4,Supermarket Type1,Tier 1,Small
5,Supermarket Type1,Tier 2,Small
6,Grocery Store,Tier 1,Small


In [15]:
df.drop(columns = 'Outlet_Size', inplace = True)

In [16]:
df = pd.merge(df, outlet_size, on = ['Outlet_Type', 'Outlet_Location_Type'], how = 'left')

In [17]:
df = df[['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales']]

In [18]:
df.Item_Fat_Content.value_counts()

Item_Fat_Content
Low Fat    6201
Regular    3525
LF          381
reg         147
low fat     129
Name: count, dtype: int64

In [19]:
# Item Fat Content

map_ = {'low fat': 0, 'lf': 0, 'regular': 1, 'reg': 1}

df['Item_Fat_Content'] = df.Item_Fat_Content.str.lower().map(map_)

In [20]:
df.Item_Fat_Content.value_counts()

Item_Fat_Content
0    6711
1    3672
Name: count, dtype: int64

In [21]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Small,Tier 1,Supermarket Type1,3735.1380
2,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
3,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
4,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,Small,Tier 1,Supermarket Type1,2097.2700


In [22]:
map_it = {'Fruits and Vegetables':    0,
'Snack Foods':               1,
'Household':                 2,
'Frozen Foods':              3,
'Dairy':                     4,
'Canned':                    5,
'Baking Goods':              6,
'Health and Hygiene':        7,
'Soft Drinks':               8,
'Meat':                      9,
'Breads':                    10,
'Hard Drinks':               11,
'Others':                    12,
'Starchy Foods':             13,
'Breakfast':                 14,
'Seafood':                   15}

df['Item_Type'] = df.Item_Type.map(map_it)

In [23]:
# df[['Item_MRP']].boxplot()
# plt.show()

In [24]:
df['Outlet_Establishment_Year'] = 2025 - df.Outlet_Establishment_Year

In [25]:
df.rename(columns = {'Outlet_Establishment_Year': 'Years_Open'}, inplace = True)

In [26]:
map_os = {'Small': 0, 'Medium': 1, 'High': 2} 

df['Outlet_Size'] = df.Outlet_Size.map(map_os)

In [27]:
df.shape

(10383, 12)

In [28]:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier', 'Years_Open',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales'],
      dtype='str')

In [29]:
df.Outlet_Type.value_counts()

Outlet_Type
Supermarket Type1    7437
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: count, dtype: int64

In [30]:
map_ot = {'Grocery Store': 0, 'Supermarket Type1': 1, 'Supermarket Type2': 2, 'Supermarket Type3': 3}

df['Outlet_Type'] = df.Outlet_Type.map(map_ot)

In [31]:
df.Outlet_Location_Type.value_counts()

Outlet_Location_Type
Tier 1    4248
Tier 3    3350
Tier 2    2785
Name: count, dtype: int64

In [32]:
map_olt = {'Tier 1': 0, 'Tier 2': 1, 'Tier 3': 2}

df['Outlet_Location_Type'] = df.Outlet_Location_Type.map(map_olt)

In [33]:
df.isna().sum()

Item_Identifier           0
Item_Weight               4
Item_Fat_Content          0
Item_Visibility           0
Item_Type                 0
Item_MRP                  0
Outlet_Identifier         0
Years_Open                0
Outlet_Size             555
Outlet_Location_Type      0
Outlet_Type               0
Item_Outlet_Sales         0
dtype: int64

In [34]:
X = df.iloc[:, ~df.columns.isin(['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'])]

In [35]:
X.isna().sum()

Item_Weight               4
Item_Fat_Content          0
Item_Visibility           0
Item_Type                 0
Item_MRP                  0
Years_Open                0
Outlet_Size             555
Outlet_Location_Type      0
Outlet_Type               0
dtype: int64

In [36]:
y = df.Item_Outlet_Sales

In [37]:
#train_rows_, test_rows_ = rand_ <= .7, rand_ > .7

In [38]:
#X_train, X_test, y_train, y_test = X[train_rows_], X[test_rows_], y[train_rows_], y[test_rows_]

In [39]:
dtrain = xgb.DMatrix(X, y)

In [40]:
params = {'learning_rate': .05,
          'max_depth': 6,
          #'subsample': .7,
          #'colsample_bytree': .7
         }

num_boost_round = 100

In [41]:
reg = xgb.train(params, dtrain, num_boost_round = num_boost_round)

In [42]:
filename = 'sales_pred_model.pkl'
with open(filename, 'wb') as f:
    pickle.dump(reg, f)

In [43]:
# Load the model from the pickle file
with open('sales_pred_model.pkl', 'rb') as f:
    reg = pickle.load(f)

In [45]:
pred_ = reg.predict(dtrain)

In [46]:
np.mean((y-pred_)**2)**.5

np.float64(979.0156311755317)